In [12]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification, make_regression, make_blobs
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    # Classification metrics
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss, confusion_matrix, classification_report,
    balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score,
    # Regression metrics
    mean_squared_error, mean_absolute_error, r2_score,
    explained_variance_score, median_absolute_error, mean_absolute_percentage_error,
    max_error, mean_poisson_deviance, mean_gamma_deviance,
    # Clustering metrics
    silhouette_score, calinski_harabasz_score, davies_bouldin_score,
    adjusted_rand_score, normalized_mutual_info_score, homogeneity_score,
    completeness_score, v_measure_score
)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor
)
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, Birch, SpectralClustering
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
import joblib
from functools import lru_cache
import concurrent.futures
from scipy.stats import spearmanr
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

# Try to import XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("XGBoost not available. Install with: pip install xgboost")

# Set matplotlib to non-interactive backend for faster performance
plt.switch_backend('Agg')

class ComprehensiveMLComparator:
    def __init__(self):
        self.scaler = StandardScaler()
        self.results_cache = {}
        self._init_preconfigured_models()

    def _init_preconfigured_models(self):
        """Pre-configure models for faster instantiation including XGBoost"""
        self.preconfigured_models = {
            'classification': {
                'logistic_regression': LogisticRegression(C=1.0, solver='liblinear', max_iter=1000, n_jobs=-1),
                'decision_tree': DecisionTreeClassifier(max_depth=5, random_state=42),
                'random_forest': RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1),
                'extra_trees': ExtraTreesClassifier(n_estimators=50, random_state=42, n_jobs=-1),
                'gradient_boosting': GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, random_state=42),
                'ada_boost': AdaBoostClassifier(n_estimators=50, random_state=42),
                'svm': SVC(C=1.0, kernel='rbf', random_state=42, probability=True),
                'linear_svm': LinearSVC(C=1.0, random_state=42, max_iter=1000),
                'knn': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
                'naive_bayes': GaussianNB(),
                'lda': LinearDiscriminantAnalysis(),
                'qda': QuadraticDiscriminantAnalysis(),
                'mlp': MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42),
                'sgd': SGDClassifier(random_state=42, n_jobs=-1),
                'gaussian_process': GaussianProcessClassifier(random_state=42) # Corrected this line
            },
            'regression': {
                'linear_regression': LinearRegression(n_jobs=-1),
                'ridge': Ridge(alpha=1.0, random_state=42),
                'lasso': Lasso(alpha=1.0, random_state=42),
                'elastic_net': ElasticNet(alpha=1.0, random_state=42),
                'decision_tree': DecisionTreeRegressor(max_depth=5, random_state=42),
                'random_forest': RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1),
                'extra_trees': ExtraTreesRegressor(n_estimators=50, random_state=42, n_jobs=-1),
                'gradient_boosting': GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, random_state=42),
                'ada_boost': AdaBoostRegressor(n_estimators=50, random_state=42),
                'svm': SVR(C=1.0, kernel='rbf'),
                'linear_svr': LinearSVR(C=1.0, random_state=42, max_iter=1000),
                'knn': KNeighborsRegressor(n_neighbors=5, n_jobs=-1),
                'mlp': MLPRegressor(hidden_layer_sizes=(50,), max_iter=500, random_state=42),
                'sgd': SGDRegressor(random_state=42),
                'gaussian_process': GaussianProcessRegressor(random_state=42) # Corrected this line
            },
            'clustering': {
                'kmeans': KMeans(n_clusters=3, random_state=42, n_init=10),
                'dbscan': DBSCAN(eps=0.5, min_samples=5),
                'agglomerative': AgglomerativeClustering(n_clusters=3),
                'birch': Birch(n_clusters=3, threshold=0.5),
                'spectral': SpectralClustering(n_clusters=3, random_state=42, n_jobs=-1)
            }
        }

        # Add XGBoost if available
        if XGBOOST_AVAILABLE:
            self.preconfigured_models['classification']['xgboost'] = xgb.XGBClassifier(
                n_estimators=50, learning_rate=0.1, random_state=42, n_jobs=-1,
                max_depth=3, subsample=0.8, colsample_bytree=0.8
            )
            self.preconfigured_models['regression']['xgboost'] = xgb.XGBRegressor(
                n_estimators=50, learning_rate=0.1, random_state=42, n_jobs=-1,
                max_depth=3, subsample=0.8, colsample_bytree=0.8
            )

    @lru_cache(maxsize=10)
    def generate_sample_data(self, task_type, n_samples=1000, n_features=10, random_state=42):
        """Generate sample data for demonstration with caching"""
        if task_type == "classification":
            X, y = make_classification(
                n_samples=n_samples, n_features=n_features,
                n_informative=min(8, n_features), n_redundant=max(1, n_features//5),
                n_classes=3, n_clusters_per_class=1, random_state=random_state
            )
            return X, y

        elif task_type == "regression":
            X, y = make_regression(
                n_samples=n_samples, n_features=n_features,
                noise=0.1, random_state=random_state
            )
            return X, y

        elif task_type == "clustering":
            X, y = make_blobs(
                n_samples=n_samples, n_features=n_features,
                centers=4, random_state=random_state, cluster_std=1.0
            )
            return X, y

        return None, None

    def preprocess_data(self, X, y=None, task_type="classification"):
        """Preprocess the data efficiently"""
        if task_type in ["classification", "regression"]:
            # Use a smaller test size for faster evaluation
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.15, random_state=42, stratify=y if task_type == "classification" else None
            )
            X_train = self.scaler.fit_transform(X_train)
            X_test = self.scaler.transform(X_test)
            return X_train, X_test, y_train, y_test
        else:
            X_scaled = self.scaler.fit_transform(X)
            return X_scaled, None, None, None

    def calculate_classification_metrics(self, model, X_train, X_test, y_train, y_test):
        """Calculate comprehensive classification metrics"""
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None

        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision_macro': precision_score(y_test, y_pred, average='macro', zero_division=0),
            'precision_weighted': precision_score(y_test, y_pred, average='weighted', zero_division=0),
            'recall_macro': recall_score(y_test, y_pred, average='macro', zero_division=0),
            'recall_weighted': recall_score(y_test, y_pred, average='weighted', zero_division=0),
            'f1_macro': f1_score(y_test, y_pred, average='macro', zero_division=0),
            'f1_weighted': f1_score(y_test, y_pred, average='weighted', zero_division=0),
            'matthews_corrcoef': matthews_corrcoef(y_test, y_pred),
            'cohen_kappa': cohen_kappa_score(y_test, y_pred),
        }

        if y_pred_proba is not None and len(np.unique(y_test)) == 2:
            metrics['roc_auc'] = roc_auc_score(y_test, y_pred_proba[:, 1])
            metrics['log_loss'] = log_loss(y_test, y_pred_proba)
        elif y_pred_proba is not None:
            metrics['roc_auc_ovr'] = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
            metrics['roc_auc_ovo'] = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
            metrics['log_loss'] = log_loss(y_test, y_pred_proba)

        # Cross-validation scores
        cv_accuracy = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy', n_jobs=-1)
        cv_f1 = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_weighted', n_jobs=-1)

        metrics['cv_accuracy_mean'] = np.mean(cv_accuracy)
        metrics['cv_accuracy_std'] = np.std(cv_accuracy)
        metrics['cv_f1_mean'] = np.mean(cv_f1)
        metrics['cv_f1_std'] = np.std(cv_f1)

        return metrics

    def calculate_regression_metrics(self, model, X_train, X_test, y_train, y_test):
        """Calculate comprehensive regression metrics"""
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        metrics = {
            'mse': mean_squared_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'mae': mean_absolute_error(y_test, y_pred),
            'mape': mean_absolute_percentage_error(y_test, y_pred),
            'r2': r2_score(y_test, y_pred),
            'explained_variance': explained_variance_score(y_test, y_pred),
            'median_ae': median_absolute_error(y_test, y_pred),
            'max_error': max_error(y_test, y_pred),
        }

        # Additional metrics if suitable
        if np.all(y_test > 0) and np.all(y_pred > 0):
            try:
                metrics['mean_poisson_deviance'] = mean_poisson_deviance(y_test, y_pred)
                metrics['mean_gamma_deviance'] = mean_gamma_deviance(y_test, y_pred)
            except:
                pass

        # Cross-validation scores
        cv_r2 = cross_val_score(model, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)
        cv_mae = cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)

        metrics['cv_r2_mean'] = np.mean(cv_r2)
        metrics['cv_r2_std'] = np.std(cv_r2)
        metrics['cv_mae_mean'] = -np.mean(cv_mae)
        metrics['cv_mae_std'] = np.std(cv_mae)

        # Spearman correlation
        metrics['spearman_corr'] = spearmanr(y_test, y_pred)[0]

        return metrics

    def calculate_clustering_metrics(self, model, X, true_labels=None):
        """Calculate comprehensive clustering metrics"""
        labels = model.fit_predict(X)

        metrics = {
            'silhouette_score': silhouette_score(X, labels) if len(np.unique(labels)) > 1 else -1,
            'calinski_harabasz_score': calinski_harabasz_score(X, labels) if len(np.unique(labels)) > 1 else -1,
            'davies_bouldin_score': davies_bouldin_score(X, labels) if len(np.unique(labels)) > 1 else float('inf'),
        }

        if true_labels is not None:
            metrics.update({
                'adjusted_rand_score': adjusted_rand_score(true_labels, labels),
                'normalized_mutual_info': normalized_mutual_info_score(true_labels, labels),
                'homogeneity_score': homogeneity_score(true_labels, labels),
                'completeness_score': completeness_score(true_labels, labels),
                'v_measure_score': v_measure_score(true_labels, labels),
            })

        return metrics, labels

    def evaluate_model(self, model, X_train, X_test, y_train, y_test, task_type):
        """Evaluate a single model with timing"""
        start_time = time.time()

        try:
            if task_type == "classification":
                metrics = self.calculate_classification_metrics(model, X_train, X_test, y_train, y_test)
            elif task_type == "regression":
                metrics = self.calculate_regression_metrics(model, X_train, X_test, y_train, y_test)
            else:  # clustering
                metrics, labels = self.calculate_clustering_metrics(model, X_train, y_train)

            execution_time = time.time() - start_time
            metrics['execution_time'] = execution_time

            return metrics, None

        except Exception as e:
            execution_time = time.time() - start_time
            return {'execution_time': execution_time}, str(e)

    def evaluate_models_parallel(self, models, X_train, X_test, y_train, y_test, task_type):
        """Evaluate multiple models in parallel"""
        results = {}

        with concurrent.futures.ThreadPoolExecutor(max_workers=min(4, len(models))) as executor:
            # Submit all tasks
            future_to_model = {
                executor.submit(
                    self.evaluate_model,
                    model, X_train, X_test, y_train, y_test, task_type
                ): name for name, model in models.items()
            }

            # Collect results as they complete
            for future in concurrent.futures.as_completed(future_to_model):
                model_name = future_to_model[future]
                try:
                    metrics, error = future.result()
                    results[model_name] = {'metrics': metrics, 'error': error}
                except Exception as e:
                    results[model_name] = {'metrics': {}, 'error': str(e)}

        return results

    def compare_algorithms(self, task_type, selected_algorithms, X, y=None):
        """Compare multiple algorithms efficiently"""
        cache_key = f"{task_type}_{hash(str(X))}_{hash(str(y)) if y is not None else 0}"

        # Check cache first
        if cache_key in self.results_cache:
            return self.results_cache[cache_key]

        # Preprocess data
        if task_type in ["classification", "regression"]:
            X_train, X_test, y_train, y_test = self.preprocess_data(X, y, task_type)
        else:
            X_train, _, _, _ = self.preprocess_data(X, task_type=task_type)
            X_test, y_train, y_test = None, None, None

        # Prepare models
        models_to_evaluate = {}
        for algo in selected_algorithms:
            if algo in self.preconfigured_models[task_type]:
                models_to_evaluate[algo] = self.preconfigured_models[task_type][algo]

        # Evaluate models in parallel
        results = self.evaluate_models_parallel(
            models_to_evaluate, X_train, X_test, y_train, y_test, task_type
        )

        # Cache results
        self.results_cache[cache_key] = results

        return results

# Create the comparator instance
comparator = ComprehensiveMLComparator()

# Create Gradio interface
def create_interface():
    with gr.Blocks(title="Comprehensive ML Algorithm Comparator with XGBoost", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# ⚡ Comprehensive ML Algorithm Comparator with XGBoost")
        gr.Markdown("Compare different ML algorithms with extensive metrics and find the best one for your dataset!")

        # Display XGBoost availability status
        if not XGBOOST_AVAILABLE:
            gr.Markdown("""
            <div style='background-color: #fff3cd; border-left: 4px solid #ffc107; padding: 10px; margin: 10px 0;'>
            ⚠️ <b>XGBoost not available</b>. Install with: <code>pip install xgboost</code> for enhanced performance.
            </div>
            """)
        else:
            gr.Markdown("""
            <div style='background-color: #d4edda; border-left: 4px solid #28a745; padding: 10px; margin: 10px 0;'>
            ✅ <b>XGBoost available</b> and ready to use for classification and regression tasks.
            </div>
            """)

        with gr.Row():
            with gr.Column(scale=1):
                task_type = gr.Dropdown(
                    choices=["classification", "regression", "clustering"],
                    label="Task Type",
                    value="classification"
                )

                use_sample_data = gr.Checkbox(label="Use Sample Data", value=True)

                with gr.Row():
                    n_samples = gr.Slider(minimum=100, maximum=5000, value=1000, step=100, label="Samples")
                    n_features = gr.Slider(minimum=2, maximum=50, value=10, step=1, label="Features")

                file_upload = gr.File(
                    label="Upload your dataset (CSV)",
                    file_types=[".csv"]
                )

                gr.Markdown("### Select Algorithms")

                # Algorithm selection based on task type
                algorithm_selection = gr.CheckboxGroup(
                    label="Algorithms",
                    choices=list(comparator.preconfigured_models['classification'].keys()),
                    value=list(comparator.preconfigured_models['classification'].keys())[:4],
                    interactive=True
                )

                # Update algorithm choices when task type changes
                def update_algorithm_choices(task_type):
                    algorithms = list(comparator.preconfigured_models[task_type].keys())
                    # Prioritize XGBoost in default selection if available
                    default_algorithms = algorithms[:3]
                    if XGBOOST_AVAILABLE and 'xgboost' in algorithms and 'xgboost' not in default_algorithms:
                        default_algorithms = ['xgboost'] + default_algorithms[:2]
                    return gr.CheckboxGroup(choices=algorithms, value=default_algorithms)

                task_type.change(
                    update_algorithm_choices,
                    inputs=task_type,
                    outputs=algorithm_selection
                )

                # XGBoost parameter tuning (when available and selected)
                with gr.Accordion("XGBoost Parameters (Advanced)", open=False) as xgb_params:
                    xgb_learning_rate = gr.Slider(0.01, 0.3, value=0.1, step=0.01, label="Learning Rate")
                    xgb_max_depth = gr.Slider(1, 10, value=3, step=1, label="Max Depth")
                    xgb_subsample = gr.Slider(0.5, 1.0, value=0.8, step=0.1, label="Subsample Ratio")
                    xgb_colsample = gr.Slider(0.5, 1.0, value=0.8, step=0.1, label="Column Sample Ratio")

                compare_btn = gr.Button("Compare Algorithms", variant="primary", size="lg")

            with gr.Column(scale=2):
                output_message = gr.Textbox(label="Status", interactive=False)

                with gr.Tab("Results Table"):
                    results_df = gr.Dataframe(
                        label="Comparison Results",
                        headers=["Algorithm", "Primary Metric", "Secondary Metric", "Time", "Status"],
                        interactive=False,
                        wrap=True
                    )

                with gr.Tab("Detailed Metrics"):
                    detailed_metrics = gr.Dataframe(
                        label="Detailed Metrics",
                        interactive=False,
                        wrap=True
                    )

                with gr.Tab("Visualization"):
                    plot_output = gr.Plot(label="Performance Comparison")

                with gr.Tab("Feature Importance") as feature_importance_tab:
                    feature_importance_plot = gr.Plot(label="Feature Importance (Top Algorithms)")

                best_algorithm = gr.Textbox(label="Recommended Algorithm", interactive=False)

        # Function to update XGBoost parameters
        def update_xgboost_parameters(learning_rate, max_depth, subsample, colsample):
            if XGBOOST_AVAILABLE:
                # Update classification XGBoost
                if 'xgboost' in comparator.preconfigured_models['classification']:
                    comparator.preconfigured_models['classification']['xgboost'] = xgb.XGBClassifier(
                        n_estimators=50, learning_rate=learning_rate, random_state=42, n_jobs=-1,
                        max_depth=max_depth, subsample=subsample, colsample_bytree=colsample
                    )

                # Update regression XGBoost
                if 'xgboost' in comparator.preconfigured_models['regression']:
                    comparator.preconfigured_models['regression']['xgboost'] = xgb.XGBRegressor(
                        n_estimators=50, learning_rate=learning_rate, random_state=42, n_jobs=-1,
                        max_depth=max_depth, subsample=subsample, colsample_bytree=colsample
                    )

            return "XGBoost parameters updated successfully!"

        # Connect XGBoost parameter updates
        for param in [xgb_learning_rate, xgb_max_depth, xgb_subsample, xgb_colsample]:
            param.change(
                update_xgboost_parameters,
                inputs=[xgb_learning_rate, xgb_max_depth, xgb_subsample, xgb_colsample],
                outputs=output_message
            )

        # Function to process the comparison
        def run_comparison(task_type, algorithm_selection, file_upload, use_sample_data, n_samples, n_features):
            start_time = time.time()

            # Get the data
            if use_sample_data:
                X, y = comparator.generate_sample_data(task_type, n_samples, n_features)
                data_source = f"Generated {task_type} data with {n_samples} samples and {n_features} features"
            elif file_upload is not None:
                try:
                    df = pd.read_csv(file_upload.name)
                    if task_type == "clustering":
                        X = df.values
                        y = None
                    else:
                        X = df.iloc[:, :-1].values
                        y = df.iloc[:, -1].values
                    data_source = f"Uploaded data with {X.shape[0]} samples and {X.shape[1]} features"
                except Exception as e:
                    return f"Error loading file: {str(e)}", None, None, None, None, None
            else:
                return "Please upload a file or use sample data", None, None, None, None, None

            # Run comparison
            results = comparator.compare_algorithms(task_type, algorithm_selection, X, y)

            # Process results for summary table
            results_list = []
            detailed_metrics_list = []
            best_score = -float('inf')
            best_algo = ""

            # For feature importance
            feature_importance_data = {}

            for algo, result in results.items():
                if result['error']:
                    results_list.append([algo, "Error", "N/A", "N/A", result['error']])
                    detailed_metrics_list.append({"Algorithm": algo, "Status": "Error", "Error": result['error']})
                else:
                    metrics = result['metrics']

                    # Determine the best metrics based on task type
                    if task_type == "classification":
                        primary_metric = f"Accuracy: {metrics.get('accuracy', 0):.4f}"
                        secondary_metric = f"F1: {metrics.get('f1_weighted', 0):.4f}"
                        score = metrics.get('accuracy', 0)
                    elif task_type == "regression":
                        primary_metric = f"R²: {metrics.get('r2', 0):.4f}"
                        secondary_metric = f"MAE: {metrics.get('mae', 0):.4f}"
                        score = metrics.get('r2', 0)
                    else:  # clustering
                        primary_metric = f"Silhouette: {metrics.get('silhouette_score', 0):.4f}"
                        secondary_metric = f"CH: {metrics.get('calinski_harabasz_score', 0):.4f}"
                        score = metrics.get('silhouette_score', 0)

                    results_list.append([
                        algo,
                        primary_metric,
                        secondary_metric,
                        f"{metrics.get('execution_time', 0):.3f}s",
                        "Success"
                    ])

                    # Add to detailed metrics
                    detailed_row = {"Algorithm": algo, "Status": "Success"}
                    for k, v in metrics.items():
                        if isinstance(v, (int, float)):
                            detailed_row[k] = round(v, 6)
                        else:
                            detailed_row[k] = v
                    detailed_metrics_list.append(detailed_row)

                    # Track best algorithm
                    if score > best_score:
                        best_score = score
                        best_algo = algo

            # Create performance visualization
            fig, ax = plt.subplots(figsize=(12, 8))

            if results_list and any("Error" not in row for row in results_list):
                # Extract data for plotting
                algorithms = [row[0] for row in results_list if row[4] != "Error"]

                if task_type == "classification":
                    scores1 = [float(row[1].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    scores2 = [float(row[2].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    ylabel1, ylabel2 = "Accuracy", "F1 Score"
                elif task_type == "regression":
                    scores1 = [float(row[1].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    scores2 = [float(row[2].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    ylabel1, ylabel2 = "R² Score", "MAE"
                else:  # clustering
                    scores1 = [float(row[1].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    scores2 = [float(row[2].split(": ")[1]) for row in results_list if row[4] != "Error"]
                    ylabel1, ylabel2 = "Silhouette Score", "Calinski-Harabasz Score"

                # Create subplots
                x = np.arange(len(algorithms))
                width = 0.35

                # First metric
                bars1 = ax.bar(x - width/2, scores1, width, label=ylabel1, alpha=0.8)
                # Second metric (inverted if needed for better visualization)
                if task_type == "regression":  # For regression, MAE is better when lower
                    scores2 = [-s for s in scores2]  # Invert for visualization
                    ylabel2 = "MAE (inverted)"

                bars2 = ax.bar(x + width/2, scores2, width, label=ylabel2, alpha=0.8)

                ax.set_ylabel('Scores')
                ax.set_title('Algorithm Performance Comparison')
                ax.set_xticks(x)
                ax.set_xticklabels(algorithms, rotation=45, ha='right')
                ax.legend()

                # Add value labels on bars
                for bars, scores in zip([bars1, bars2], [scores1, scores2]):
                    for bar, score in zip(bars, scores):
                        height = bar.get_height()
                        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                                f'{score:.3f}', ha='center', va='bottom', fontsize=8)

                plt.tight_layout()
            else:
                ax.text(0.5, 0.5, 'No results to display',
                        ha='center', va='center', transform=ax.transAxes)
                ax.set_title('Algorithm Performance Comparison')

            # Create feature importance plot for tree-based models
            fig_importance, ax_importance = plt.subplots(figsize=(12, 8))

            if task_type in ["classification", "regression"] and X is not None:
                try:
                    # Get feature names or create generic ones
                    if hasattr(X, 'columns'):
                        feature_names = X.columns.tolist()
                    else:
                        feature_names = [f'Feature_{i}' for i in range(X.shape[1])]

                    # Check for tree-based models that have feature importance
                    tree_models = []
                    for algo in algorithm_selection:
                        if algo in results and not results[algo]['error']:
                            model = comparator.preconfigured_models[task_type][algo]
                            if hasattr(model, 'feature_importances_'):
                                # Train the model to get feature importance
                                X_train, X_test, y_train, y_test = comparator.preprocess_data(X, y, task_type)
                                model.fit(X_train, y_train)
                                importance = model.feature_importances_
                                tree_models.append((algo, importance))

                    if tree_models:
                        # Plot feature importance for top models
                        n_models = min(3, len(tree_models))
                        fig_importance, axes = plt.subplots(n_models, 1, figsize=(12, 4 * n_models))

                        if n_models == 1:
                            axes = [axes]

                        for i, (algo, importance) in enumerate(tree_models[:n_models]):
                            # Sort features by importance
                            indices = np.argsort(importance)[::-1]
                            sorted_importance = importance[indices]
                            sorted_features = [feature_names[j] for j in indices]

                            # Plot top 10 features
                            top_n = min(10, len(sorted_importance))
                            axes[i].barh(range(top_n), sorted_importance[:top_n][::-1], align='center')
                            axes[i].set_yticks(range(top_n))
                            axes[i].set_yticklabels(sorted_features[:top_n][::-1])
                            axes[i].set_xlabel('Feature Importance')
                            axes[i].set_title(f'Feature Importance - {algo}')

                        plt.tight_layout()
                    else:
                        ax_importance.text(0.5, 0.5, 'No feature importance data available\n(Tree-based models only)',
                                         ha='center', va='center', transform=ax_importance.transAxes)
                        ax_importance.set_title('Feature Importance')
                except Exception as e:
                    ax_importance.text(0.5, 0.5, f'Error generating feature importance: {str(e)}',
                                     ha='center', va='center', transform=ax_importance.transAxes)
                    ax_importance.set_title('Feature Importance')
            else:
                ax_importance.text(0.5, 0.5, 'Feature importance not available for clustering',
                                 ha='center', va='center', transform=ax_importance.transAxes)
                ax_importance.set_title('Feature Importance')

            total_time = time.time() - start_time

            return (
                f"{data_source}. Comparison completed in {total_time:.2f} seconds.",
                results_list,
                pd.DataFrame(detailed_metrics_list),
                fig,
                fig_importance,
                f"{best_algo} (Score: {best_score:.4f})"
            )

        # Set up event handler
        compare_btn.click(
            fn=run_comparison,
            inputs=[task_type, algorithm_selection, file_upload, use_sample_data, n_samples, n_features],
            outputs=[output_message, results_df, detailed_metrics, plot_output, feature_importance_plot, best_algorithm]
        )

        # Examples for quick testing
        gr.Markdown("### Quick Examples")
        with gr.Row():
            gr.Examples(
                examples=[
                    ["classification", ["xgboost", "random_forest", "gradient_boosting"], True, 1000, 10],
                    ["regression", ["xgboost", "random_forest", "gradient_boosting"], True, 1000, 10],
                    ["clustering", ["kmeans", "agglomerative", "birch"], True, 1000, 10]
                ],
                inputs=[task_type, algorithm_selection, use_sample_data, n_samples, n_features],
                label="Click any example to quickly set up the interface"
            )

    return demo

# Create and launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3049386ad897d385d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
